In [7]:
import numpy as np
import pickle
import os
import random

from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score,accuracy_score
import matplotlib.pyplot as plt
import itertools
from sklearn import metrics

import tensorflow as tf
import torch
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
from torch.utils.data import TensorDataset

from ParticipantLab import ParticipantLab as parti
from copied_from_guihong_server.models import DeepConvLSTM_Split

In [8]:
P = 15
win_size = 10
hop = .5

participants = []

# prepare user data
PATH_data = '/media/hd4t1/dawei/audioimu/lab'

if os.path.exists(PATH_data + '/rawAudioSegmentedData_window_' + str(win_size) + '_hop_' + str(hop) + '_Test_NEW.pkl'):
    with open(PATH_data + '/rawAudioSegmentedData_window_' + str(win_size) + '_hop_' + str(hop) + '_Test_NEW.pkl', 'rb') as f:
        participants = pickle.load(f)


# load user data
window_size = 1024
hop_size = 320
batch_size = 64  
model_name =  'DeepConvLSTM_Split' 
fmin, fmax = 50, 11000
mel_bins = 64
classes_num = 23
sr = 22050
learning_rate = 1e-4
num_epochs = 200
device = 'cuda'
sub_list = np.arange(15)

print("# of participants:", len(sub_list), len(participants))

# of participants: 15 15


In [9]:
overall_acc, overall_f1 = [], []
test_oo_overall, true_test_oo_overall = [], []

for sub in sub_list:
    # load training set
    X_trainM = np.empty((0,np.shape(participants[0].rawMdataX_s1)[1], np.shape(participants[0].rawMdataX_s1)[-1]))
    X_trainA = np.empty((0,np.shape(participants[0].rawAdataX_s1)[-1]))
    y_train = np.zeros((0, 1))
        
    # load val set
    X_testM = np.empty((0,np.shape(participants[0].rawMdataX_s1)[1], np.shape(participants[0].rawMdataX_s1)[-1]))
    X_testA = np.empty((0,np.shape(participants[0].rawAdataX_s1)[-1]))
    y_test = np.zeros((0, 1))
        
    for u in [participants[sub]]:
        print("test participant (lopo): " + u.name)
        #print(X_testM.shape)
        X_testM = np.vstack((X_testM, u.rawMdataX_s1[:]))
        X_testM = np.vstack((X_testM, u.rawMdataX_s2[:]))
        X_testA = np.vstack((X_testA, u.rawAdataX_s1[:]))
        X_testA = np.vstack((X_testA, u.rawAdataX_s2[:]))
        y_test = np.vstack((y_test, u.rawdataY_s1))
        y_test = np.vstack((y_test, u.rawdataY_s2))
        #print(X_testM.shape, X_testA.shape, y_test.shape)
        #y_test = np.vstack((u.rawdataY_s1, u.rawdataY_s2))
    
    # filter out NULL
    y_test = y_test.flatten()
    X_testM = X_testM[y_test != 23]
    X_testA = X_testA[y_test != 23]
    y_test = y_test[y_test != 23]	
    
    y_train = y_train.flatten()
    X_trainM = X_trainM[y_train != 23]
    X_trainA = X_trainA[y_train != 23]	
    y_train = y_train[y_train != 23]
    
    # one hot
    y_test = y_test.astype('int64')
    y_train = y_train.astype('int64')

    torch.cuda.empty_cache()
    
    Model = eval(model_name)
    
    # CNN
    model = Model(classes_num=classes_num, acc_features=np.shape(X_trainM[:,:,:3])[-1], gyr_features = np.shape(X_trainM[:,:,3:])[-1])
    
    # Parallel
    print('GPU number: {}'.format(torch.cuda.device_count()))
    model = torch.nn.DataParallel(model)
    
    if 'cuda' in str(device):
        model.to(device)
    
    pretrained_checkpoint_path = './copied_from_guihong_server/IMU_models/DeepConvLSTM_Split_NEW_kd_expr2 (teacher2)/participant_%s' % u.name
    checkpoint_model = [os.path.join(pretrained_checkpoint_path, i) for i in os.listdir(pretrained_checkpoint_path) if i.endswith('.pth')][0]
    reported_acc, reported_f1 = checkpoint_model.split('=')[1][:6], checkpoint_model.split('=')[2][:6]
    reported_acc, reported_f1 = float(reported_acc), float(reported_f1)

    checkpoint = torch.load(checkpoint_model, map_location='cuda')
    model.load_state_dict(checkpoint)
    model.eval()
    
    x_trainM_tensor = torch.from_numpy(np.array(X_trainM)).float()
    x_trainA_tensor = torch.from_numpy(np.array(X_trainA)).float()
    y_train_tensor = torch.from_numpy(np.array(y_train)).float()
    x_testM_tensor = torch.from_numpy(np.array(X_testM)).float()
    x_testA_tensor = torch.from_numpy(np.array(X_testA)).float()
    y_test_tensor = torch.from_numpy(np.array(y_test)).float()
    
    test_data = TensorDataset(x_testM_tensor, x_testA_tensor, y_test_tensor)
    
    test_loader = torch.utils.data.DataLoader(dataset=test_data, 
                            batch_size=batch_size,
                            num_workers=8, pin_memory=True, shuffle = False)
    
    eval_output = []
    true_output = []
    test_output = []
    true_test_output = []
    
    with torch.no_grad():
        for x_val, x_val_a, y_val in test_loader:
            # split acc and gyro inputs for as required by the student
            acc_val, gyr_val =  np.expand_dims(np.array(x_val[:,:,:3]), axis = 1), np.expand_dims(np.array(x_val[:,:,3:]), axis = 1)               
            acc_val = torch.from_numpy(np.array(acc_val)).float()
            acc_val = acc_val.to(device)
            gyr_val = torch.from_numpy(np.array(gyr_val)).float()
            gyr_val = gyr_val.to(device)
            x_val_a = x_val_a.to(device)
            y_val = y_val.to(device)

            yhat = model(acc_val, gyr_val)['clipwise_output']
    
            test_output.append(yhat.data.cpu().numpy())
            true_test_output.extend(y_val.data.cpu().numpy())

        test_oo = np.argmax(np.vstack(test_output), axis = 1)
        true_test_oo = np.asarray(true_test_output)

        accuracy = round(metrics.balanced_accuracy_score(true_test_oo, test_oo), 4)

        precision, recall, fscore,_ = metrics.precision_recall_fscore_support(true_test_oo, test_oo, labels=np.unique(true_test_oo), average='macro')
        fscore = round(fscore, 4)
        
    print('TEST f1: {}'.format(fscore), 'reported f1:', reported_f1)
    print('TEST acc: {}'.format(accuracy), 'reported acc:', reported_acc)
    overall_acc.append(accuracy)
    overall_f1.append(fscore)
    test_oo_overall.extend(test_oo)
    true_test_oo_overall.extend(true_test_oo)

test participant (lopo): 01
GPU number: 2


/home/dawei/anaconda3/envs/tf/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1850: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')
/home/dawei/anaconda3/envs/tf/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TEST f1: 0.6659 reported f1: 0.637
TEST acc: 0.7115 reported acc: 0.7115
test participant (lopo): 02
GPU number: 2


/home/dawei/anaconda3/envs/tf/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TEST f1: 0.649 reported f1: 0.649
TEST acc: 0.6638 reported acc: 0.6638
test participant (lopo): 03
GPU number: 2
TEST f1: 0.625 reported f1: 0.625
TEST acc: 0.6445 reported acc: 0.6445
test participant (lopo): 04
GPU number: 2
TEST f1: 0.7544 reported f1: 0.7544
TEST acc: 0.7813 reported acc: 0.7813
test participant (lopo): 05
GPU number: 2
TEST f1: 0.7333 reported f1: 0.7333
TEST acc: 0.7492 reported acc: 0.7492
test participant (lopo): 06
GPU number: 2


...

/home/dawei/anaconda3/envs/tf/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TEST f1: 0.7511 reported f1: 0.7511
TEST acc: 0.7816 reported acc: 0.7816
test participant (lopo): 14
GPU number: 2


/home/dawei/anaconda3/envs/tf/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TEST f1: 0.7117 reported f1: 0.7117
TEST acc: 0.7429 reported acc: 0.7429
test participant (lopo): 15
GPU number: 2
TEST f1: 0.6872 reported f1: 0.6872
TEST acc: 0.7115 reported acc: 0.7115


/home/dawei/anaconda3/envs/tf/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [10]:
print('overall acc, f1:', np.mean(overall_acc), np.mean(overall_f1))

overall acc, f1: 0.7395533333333333 0.7159800000000001
